In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pickle
import os
import itertools
import datetime
import os, sys
import pandas as pd

# To load personal libraries (currently in folder 'tools'):
sys.path.append('tools')

#import generic_tools_pl as gt   # like hexcolor(), ...
#import specific_nn_tools_pl as nnt  # like CustomDataset(), Net(), train_model() 
import generic_tools_pl as gt
from generic_tools_pl import region

gt.region = input("Enter the region number: ")
region=gt.region
print(gt.region)

48


In [2]:
# Call set_printoptions(threshold=x) to change the threshold where arrays are truncated. To prevent truncation, set the threshold to np.inf.
# Change linewidth to print wider lines ...
#display(np.get_printoptions())
np.set_printoptions(threshold=np.inf,linewidth=180) # good for a 1920 pixels wide screen
pd.set_option('display.max_columns', 18)            # 
#np.set_printoptions(threshold=np.inf,linewidth=240)   # good for an iMac 27" screen
#pd.set_option('display.max_columns', 100)             # (this is maybe too much!)

In [3]:
# Label identifiant la méthode de génération des données
data_gener_method = 'INVDGM1'  # Combinaisons G-A-N-H Random

#seed_for_combi = 0    # seed 0 utilisée pour TRAIN-TEST, je prefere utiliser un autre ... 
seed_for_combi = 1    # "semence" d'initialisation du générateur de chiffres aléatoires

# nombre de données (combinaisons GANH) tirées par modèle, maximum (decommenter le choix)
n_sim_max_by_mod = 100
#n_sim_max_by_mod = 400
#n_sim_max_by_mod = 1000

#--------------------
dataframe_format_ok = True
array_format_ok = False
#--------------------
save_anc_data_only_ok = True  # if TRUE saves a dictionary with index, number by mod, list of models, of forcings, of years. but not data arrays
#save_anc_data_only_ok = False
#--------------------
save_data_ok = True
#--------------------
#rewrite_files_ok = True
rewrite_files_ok = False

In [4]:
# -----------------------------------------------------------------------------------------------
# Selection of working Forcing and Models:
# -----------------------------------------------------------------------------------------------

# list of selected forcings
#forcing_names = [ 'ghg', 'aer', 'nat', 'hist' ]
#forcing_color = [ 'r',   'b',   'g',   'k' ]

forcing_names = [ 'ghg', 'aer', 'nat' ]   # 'hist' n'intervient pas dans l'ensemble de données INVERSION
forcing_color = [ 'r',   'b',   'g' ]


# list of selected models 
model_names = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1', 'CanESM5', 'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'NorESM2-LM']
#model_names = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1', 'CanESM5', 'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MRI-ESM2-0', 'NorESM2-LM']


In [5]:
#verbose = False
verbose = True
#--------------------
save_figs = True
#save_figs = False
#--------------------
figs_dir = 'Figs'

local_fig_label = "Nb1_gener_INV_data_set_to_df"   # label a appliquer aux noms des fichiers (figures) produits dans le notebook

figs_defaults = { 'dpi' : 300, 'facecolor' : 'w', 'edgecolor' : 'w'} # ajuter format='png' ou autre a l'appel de savefig()

if save_figs and not os.path.exists(figs_dir):
    os.makedirs(figs_dir)

# ether_dir = '/net/ether/data/varclim/ggalod/Constantin'

#data_dir = '/datatmp/data/constantin_data/data_source'
#data_dir = '/net/acratopotes/datatmp/data/constantin_data/data_source'
#data_dir = 'data'

# Repertoire des donnees
try:
    # WORK dir carlos projet ryn sur Jean Zay
    data_dir = "/usr/home/habbar/Bureau/data_nc/stagelong/projetlong/data_source_dr/Region{}".format(region)
    if not os.path.isdir(data_dir):
        print(f" ** data_dir '{data_dir}' not found. Trying next...")
        
        # WORK dir Guillaume sur Jean Zay ** NON **
        #data_dir = '/gpfswork/rech/ryn/rces866/Constantin'
        #if not os.path.isdir(data_dir):
        #print(f" ** data_dir '{data_dir}' not found. Trying next...")
        
        # SSD sur Acratopotes au Locean
        data_dir = '/net/acratopotes/datatmp/data/constantin_data/data_source_pl'
        if not os.path.isdir(data_dir):
            print(f" ** data_dir '{data_dir}' not found. Trying next...")
            
            # sur Cloud SU (carlos)
            data_dir = os.path.expanduser('~/Clouds/SUnextCloud/Labo/Travaux/Theses-et-stages/These_Constantin/constantin_data/data_source_pl')
            if not os.path.isdir(data_dir):
                print(f" ** data_dir '{data_dir}' not found. Trying next...")
                
                # en dernier recours, en esperant qu'il y a un repertoire 'data' present ...
                data_dir = os.path.expanduser('data')
                if not os.path.isdir(data_dir):
                    print(f" ** data_dir '{data_dir}' not found at all **\n")
                    raise Exception('data_dir not found')

except Exception as e:
    print(f'\n *** Exception error "{e}" ***\n')
    raise

print(f"data_dir found at '{data_dir}'")

#dataframe_ok = True
dataframe_ok = False

data_dir found at '/usr/home/habbar/Bureau/data_nc/stagelong/projetlong/data_source_dr/Region48'


In [6]:
def load_data_and_gener_df(models=None, forcings=None, 
                           data_dir='.', sim_file="All_sim_z{}v0.p".format(region), models_file="models_z{}v0.p".format(region), 
                           forcings_src=[ 'ghg', 'aer', 'nat', 'hist', 'other' ], verbose=False):
    print(f"\nLecture des meta donnees (ALL_sim, liste de modeles, ...) ...")

    # ALL_sim_v4 et Models_v4:
    _all_sim_src  = pickle.load( open( os.path.join(data_dir,"All_sim_z{}v0.p".format(region)), "rb" ), encoding="latin1")

    # liste de modeles
    _all_models_src  = pickle.load( open( os.path.join(data_dir,"models_z{}v0.p".format(region)), "rb" ), encoding="latin1")
    
    # forcages dans l'ordre des lignes de _all_sim_src.
    # l'ordre des forcages est trouve en comparant le nombre de simulations par forcage (tout modele)
    # avec les dimensions des donnees dans les fichiers "T_xxx_v4.p", pour les differents forcages xxx
    _all_forcings_src = forcings_src

    # list of years in source files 
    years = np.arange(1850,2015)

    if models is None :
        models = _all_models_src

    if forcings is None :
        models = forcings_src[:-1] # tous sauf le dernier

    # tableau d'indices de debut et de fin des donnees du modele (colonne) pour chaque forcage (ligne)
    simulation_ini_index_list = np.cumsum(_all_sim_src,axis=1) - _all_sim_src
    simulation_end_index_list = np.cumsum(_all_sim_src,axis=1)

    if verbose:
        print("\nContenu de 'ALL_sim (v4).p':")
        print(_all_sim_src)
        print("Dim:",_all_sim_src.shape)
        print(f'\n_all_models_src: {len(_all_models_src)} models (v4) :\n  {_all_models_src}')
        print(f'\n_all_forcings_src: {len(_all_forcings_src)} forcings :\n  {_all_forcings_src}')
        print('\n Somme par colonne:',np.sum(_all_sim_src,axis=0))  #,"    (je suppose, c'est le nombre de simulations par modele, pour les 4+1 forcages)")
        print('\n Somme par ligne:  ',np.sum(_all_sim_src,axis=1))  #,"    (je suppose, c'est le nombre de simulations par forcace, tout modele confondu)" )
        print("\nIndices INI et END des tableaux des donnees dans fichiers 'T_xxx_v4.p' ('xxx' etant le forcage):")
        print(f"Indices INI:\n{simulation_ini_index_list}")
        print(f"Indices INI:\n{simulation_end_index_list}")

    print(f"\nLecture des donnees des forcages ...")

    list_of_df = []
    for iforcing,forcing in enumerate(forcings) :
        if verbose:
            print(f" {forcing.upper()}",end='')
        kfor = _all_forcings_src.index(forcing)
        current_t_array_filename = f"T_{forcing.lower()}_z{region}v0.p"
        T_xxx  = pickle.load(open( os.path.join(data_dir,current_t_array_filename), "rb" ),
                             encoding="latin1")
        if verbose:
            #print(T_xxx)
            print(f" Data '{current_t_array_filename}' -> dim: {T_xxx.shape}")

        #current_t_df_varname = f'T_{forcing.lower()}_df'
        xxx_df = pd.DataFrame(T_xxx, columns=years)

        xxx_df['model'] = '<NA>'   # ajoute colonne 'model'
        cols = xxx_df.columns.tolist()
        xxx_df = xxx_df[cols[-1:] + cols[:-1]]    # remet la colonne 'model' en premier
        #yyy_df = xxx_df.copy()
        for imod,mod in enumerate(models) :
            kmod = _all_models_src.index(mod)
            ii = simulation_ini_index_list[kfor,kmod]
            jj = simulation_end_index_list[kfor,kmod]
            xxx_df.loc[np.arange(ii,jj),'model'] = mod
            if verbose:
                print(f"   {imod}) '{mod}' -> {ii}, {jj} ({jj-ii})")
        # retire les lignes non identifiees par un modele
        xxx_df = xxx_df.loc[lambda df: df['model'] != '<NA>', :]
        #globals()[current_t_df_varname] = xxx_df
        #print(f" ... assigned to DataFrame '{current_t_df_varname}'")
        list_of_df.append(xxx_df)
    
    return models, forcings, years, list_of_df

In [7]:
#def load_data_and_gener_df(models=None, forcings=None, 
#                           data_dir='.', sim_file="ALL_sim_v4.p", models_file="Models_v4.p", 
#                           forcings_src=[ 'ghg', 'aer', 'nat', 'hist', 'other' ], verbose=False):
#    ...
#    return models, forcings, list_of_df

_, _, years, (T_ghg_df,T_aer_df,T_nat_df) = load_data_and_gener_df(models=model_names, forcings=forcing_names, data_dir=data_dir, verbose=verbose)


Lecture des meta donnees (ALL_sim, liste de modeles, ...) ...

Contenu de 'ALL_sim (v4).p':
[[ 3  3  3  8 50  3 10  4  8  3  5  3]
 [ 3  3  2  8 30  3 10  4  8 10  5  3]
 [ 3  3  3  8 50  3 20  4  8 50  5  3]
 [15  3 11 28 65  6 28  5 31 50 11  3]]
Dim: (4, 12)

_all_models_src: 12 models (v4) :
  ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1', 'CanESM5', 'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'NorESM2-LM']

_all_forcings_src: 5 forcings :
  ['ghg', 'aer', 'nat', 'hist', 'other']

 Somme par colonne: [ 24  12  19  52 195  15  68  17  55 113  26  12]

 Somme par ligne:   [103  89 160 256]

Indices INI et END des tableaux des donnees dans fichiers 'T_xxx_v4.p' ('xxx' etant le forcage):
Indices INI:
[[  0   3   6   9  17  67  70  80  84  92  95 100]
 [  0   3   6   8  16  46  49  59  63  71  81  86]
 [  0   3   6   9  17  67  70  90  94 102 152 157]
 [  0  15  18  29  57 122 128 156 161 192 242 253]]
Indices INI:
[[  3   6   9  17  

In [8]:
display(T_ghg_df.head())
display(T_aer_df.head())
display(T_nat_df.head())
#display(T_nat_df.drop('model', axis=1).head()) # retire la colonne 'model' et affiche les 5 premieres lignes uniquement

,model,1850,1851,1852,1853,1854,1855,1856,1857,...,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,ACCESS-ESM1-5,0.437182,0.028697,-0.132890,-1.233601,0.100446,-0.244363,0.339896,-0.276080,...,0.595394,0.787810,1.554385,1.275888,1.510629,1.722107,1.376684,1.043598,1.044848
1,ACCESS-ESM1-5,-0.363110,-0.598210,-1.077751,-0.570378,-0.637022,0.532596,0.130407,-0.499797,...,1.578003,1.574267,1.542609,1.852793,1.670341,1.553944,0.930764,0.828126,0.280815
2,ACCESS-ESM1-5,-0.128842,-0.289341,0.487822,0.006407,0.398716,0.373554,0.008501,0.205370,...,1.933489,1.824191,0.732037,1.081758,1.779265,1.542962,1.575966,1.082869,2.074988
3,BCC-CSM2-MR,-0.363994,-0.363136,0.894514,-0.348102,-0.656897,0.214344,-0.628903,0.441889,...,1.367620,0.801906,1.459528,1.279894,2.122876,0.810455,0.824601,1.445404,0.898101
4,BCC-CSM2-MR,-0.083050,0.669886,0.272358,-0.170488,0.695409,-0.164629,0.360722,-0.281259,...,0.917454,1.247491,1.630314,0.561290,1.287478,1.567756,0.760183,1.370288,0.626601


,model,1850,1851,1852,1853,1854,1855,1856,1857,...,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,ACCESS-ESM1-5,0.202815,0.005839,-0.129768,-0.495416,0.630230,0.157939,0.384925,0.662635,...,-2.075051,-0.867437,-0.262349,-0.803109,-0.578032,-0.239123,-0.361775,-1.248654,0.091179
1,ACCESS-ESM1-5,-0.269628,0.252691,0.147903,-0.282876,-0.156279,-0.083549,-0.276571,-0.082585,...,-1.215785,-0.220827,-0.633552,-0.282613,0.437398,-0.419917,-0.893562,-0.437698,-0.686877
2,ACCESS-ESM1-5,0.262341,0.043966,0.431122,0.581800,-0.320898,-0.680623,0.320839,-0.231924,...,-0.817986,-0.962634,-0.040638,0.145956,-0.674766,-0.941872,-0.633496,-0.586338,-0.623554
3,BCC-CSM2-MR,0.374439,-0.093552,0.141826,0.030901,0.227480,-0.092903,0.292051,0.293260,...,0.246638,-0.908039,0.049500,-0.578767,-0.004509,0.126799,-0.206439,0.031136,-0.132799
4,BCC-CSM2-MR,-0.193991,-0.161548,0.409424,0.173279,0.247638,-0.027724,0.067084,0.071564,...,-0.627309,-0.005084,0.521809,-0.577788,0.011284,-0.039107,-0.233216,-0.460674,0.018744


,model,1850,1851,1852,1853,1854,1855,1856,1857,...,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,ACCESS-ESM1-5,0.227257,-0.525969,-0.895317,0.299532,-0.095565,-0.774033,0.069843,-0.100081,...,-0.132396,0.788739,0.191328,-0.333537,0.624169,0.482351,0.346880,0.195535,-0.039234
1,ACCESS-ESM1-5,-0.076440,-0.440457,-0.546192,0.683881,0.136328,-0.367491,0.670634,0.096443,...,-0.050907,0.104109,-0.139384,0.365154,0.426976,-0.288740,0.446080,0.311576,-0.650231
2,ACCESS-ESM1-5,-0.347524,0.107157,0.337798,-0.303354,0.712290,0.148228,-0.074081,0.693874,...,0.016079,-0.002354,0.512459,-0.327096,0.074828,0.134329,-0.001549,0.520486,-0.171999
3,BCC-CSM2-MR,-0.016728,0.308265,-0.393468,-0.136529,-0.415972,-0.110526,-0.013817,0.284995,...,0.482887,-0.767081,-0.585095,0.534907,0.136004,0.374640,0.483726,-0.070630,0.441836
4,BCC-CSM2-MR,-0.459098,0.568948,-0.334487,-0.317023,0.786975,-0.484922,0.439867,-0.469412,...,-0.787866,-0.088455,0.374357,-0.031611,-0.166075,-0.068990,0.502052,-0.212527,0.273807


In [9]:
%%time

#verbose = False

# Initialisation des listes retournées par la boucle ---------------------------------------------------------------------
inversion_combi_list  = []   # liste des indices pour composer l'ensemble pour l'INVERSION. 
#
n_inversion_combi_list  = [] # liste du nombre de données pour l'INVERSION par modèle (le nombre de combinaisons GANH limité au max par la valeur dans la variable n_sim_max_by_mod)
# ------------------------------------------------------------------------------------------------------------------------

for imod,mod in enumerate(model_names) :
    #imod = 0
    #imod = 5
    #mod = model_names[imod]

    if verbose :
        print(f"\n{'-'*92}",end='')
    print(f'\n{imod:2d}) {mod:15s} ',end='')

    # initialise le generateur de nombres aléatoires
    print(f"\n Re-initialise le generateur de nombres aleatoires par le SEED: {seed_for_combi}")
    np.random.seed(seed_for_combi)

    # liste de simulations par forcage du modele en cours:
    T_mod_isim_by_forcing = []
    for iforcing,forcing in enumerate(forcing_names) :
        current_t_df_varname = f'T_{forcing.lower()}_df'
        xxx_df = globals()[current_t_df_varname]
        xxc_indices = xxx_df.loc[lambda df: df['model'] == mod].index
        ii = xxc_indices[0]    # indice inferieur (incluyant)
        jj = xxc_indices[-1]+1 # indice superieur (excluyant)
        if verbose:
            print('\n  ',iforcing, forcing, ii, jj, jj-ii,end='')

        # cumule toutes les simulations du modele et forcage en cours
        T_mod_isim_by_forcing.append(np.arange(ii,jj).tolist())
    
    if verbose :
        print(f'\n  Indices des focages GAN (GHG, AER, NAT):')
        [print(f"\n    {len(ksim)} indices pour {forcing_names[ii].upper()}: {ksim}",end='') for ii,ksim in enumerate(T_mod_isim_by_forcing)];
    else:
        print(f'\n  Indices des focages ',end='')
        [print(f"  {len(ksim)} pour {forcing_names[ii].upper()},",end='') for ii,ksim in enumerate(T_mod_isim_by_forcing)];
        print('')

    all_forcings_combi = list(itertools.product(*T_mod_isim_by_forcing))
    n_all_forcings_combi = len(all_forcings_combi)
    
    if verbose :
        n_to_show = int(np.min([n_all_forcings_combi/2, 9]))
        print(f"\n  {n_all_forcings_combi} combinaisons d'indices GAN:")
        print('   ',*all_forcings_combi[:n_to_show], ' ...\n     ', *all_forcings_combi[-n_to_show:])
    
    # ------------------------------------------------------------------------------------------------------------------------
    # INVERSION Set:
    # Nombre variable de patterns (combinaisons G-A-N-H) par modèle. Pas de repetitions. Toutes les combinaisons G-A-N-H sont
    # prise en compte mais avec un nombre max limite. Combinaisons choisies aleatoirement uniquement dans le cas ou il faut
    # restreindre leur nombre.
    # ------------------------------------------------------------------------------------------------------------------------

    if n_all_forcings_combi > n_sim_max_by_mod :
        permuted_inversion_all_combi = np.random.permutation(all_forcings_combi).tolist()
        inversion_all_combi = permuted_inversion_all_combi[:n_sim_max_by_mod]
    
        print(f"\n ** Restricting to {len(inversion_all_combi)} over {n_all_forcings_combi} sim. combinations for INVERSION set for {mod}.")
    else:
        inversion_all_combi = all_forcings_combi
        print(f"\n  - Selecting all {len(inversion_all_combi)} sim. combinations for INVERSION set for {mod}.")
    
    inversion_combi_list += inversion_all_combi
    n_inversion_combi_list += [len(inversion_all_combi)]


--------------------------------------------------------------------------------------------
 0) ACCESS-ESM1-5   
 Re-initialise le generateur de nombres aleatoires par le SEED: 1

   0 ghg 0 3 3
   1 aer 0 3 3
   2 nat 0 3 3
  Indices des focages GAN (GHG, AER, NAT):

    3 indices pour GHG: [0, 1, 2]
    3 indices pour AER: [0, 1, 2]
    3 indices pour NAT: [0, 1, 2]
  27 combinaisons d'indices GAN:
    (0, 0, 0) (0, 0, 1) (0, 0, 2) (0, 1, 0) (0, 1, 1) (0, 1, 2) (0, 2, 0) (0, 2, 1) (0, 2, 2)  ...
      (2, 0, 0) (2, 0, 1) (2, 0, 2) (2, 1, 0) (2, 1, 1) (2, 1, 2) (2, 2, 0) (2, 2, 1) (2, 2, 2)

  - Selecting all 27 sim. combinations for INVERSION set for ACCESS-ESM1-5.

--------------------------------------------------------------------------------------------
 1) BCC-CSM2-MR     
 Re-initialise le generateur de nombres aleatoires par le SEED: 1

   0 ghg 3 6 3
   1 aer 3 6 3
   2 nat 3 6 3
  Indices des focages GAN (GHG, AER, NAT):

    3 indices pour GHG: [3, 4, 5]
    3 indices pou


 ** Restricting to 100 over 75000 sim. combinations for INVERSION set for CanESM5.

--------------------------------------------------------------------------------------------
 5) FGOALS-g3       
 Re-initialise le generateur de nombres aleatoires par le SEED: 1

   0 ghg 67 70 3
   1 aer 46 49 3
   2 nat 67 70 3
  Indices des focages GAN (GHG, AER, NAT):

    3 indices pour GHG: [67, 68, 69]
    3 indices pour AER: [46, 47, 48]
    3 indices pour NAT: [67, 68, 69]
  27 combinaisons d'indices GAN:
    (67, 46, 67) (67, 46, 68) (67, 46, 69) (67, 47, 67) (67, 47, 68) (67, 47, 69) (67, 48, 67) (67, 48, 68) (67, 48, 69)  ...
      (69, 46, 67) (69, 46, 68) (69, 46, 69) (69, 47, 67) (69, 47, 68) (69, 47, 69) (69, 48, 67) (69, 48, 68) (69, 48, 69)

  - Selecting all 27 sim. combinations for INVERSION set for FGOALS-g3.

--------------------------------------------------------------------------------------------
 6) GISS-E2-1-G     
 Re-initialise le generateur de nombres aleatoires par le 

In [10]:
# Affichage plot pour les 3 premiers modèles
if verbose :
    print(f"\n\n{'-'*92}")
    print(f"\nINVERSION Set:")
    print(f"\n  {len(inversion_combi_list)} combinaisons d'indices GAN:")
    print('   ',*inversion_combi_list[:n_to_show], ' ...\n     ', *inversion_combi_list[-n_to_show:])
    print(f"\n  {len(n_inversion_combi_list)} nombres de combinaisons d'indices GAN par modèle:")
    print('   ',*n_inversion_combi_list[:n_to_show], ' ...\n     ', *n_inversion_combi_list[-n_to_show:])



--------------------------------------------------------------------------------------------

INVERSION Set:

  790 combinaisons d'indices GAN:
    (0, 0, 0) (0, 0, 1) (0, 0, 2) (0, 1, 0) (0, 1, 1) (0, 1, 2) (0, 2, 0) (0, 2, 1) (0, 2, 2)  ...
      (102, 86, 157) (102, 86, 158) (102, 86, 159) (102, 87, 157) (102, 87, 158) (102, 87, 159) (102, 88, 157) (102, 88, 158) (102, 88, 159)

  12 nombres de combinaisons d'indices GAN par modèle:
    27 27 18 100 100 27 100 64 100  ...
      100 100 27 100 64 100 100 100 27


In [11]:
#inversion_combi_list
n_inversion_combi_list

[27, 27, 18, 100, 100, 27, 100, 64, 100, 100, 100, 27]

In [12]:
# conversion en array d'indices les listes
inversion_combi_index = np.array(inversion_combi_list)    # indices pour composer l'ensemble d'INVERSION. 
#
n_inversion_combi = n_inversion_combi_list       # nombre de données en INVERSION par modèle (le nombre de combinaisons GANH limité au max par la valeur dans la variable n_sim_max_by_mod)

In [13]:
def save_data_file(data, filename, data_dir='.', label='data', rewrite=False, df=False):
                   
    current_filename = os.path.join(data_dir,filename)

    if (os.path.isfile(current_filename) and rewrite) or not os.path.isfile(current_filename) :
        if os.path.isfile(current_filename) :
            print(f" Saving current {label} to filename '{filename}' ... ** File exists: REWRITED **")
        else:
            print(f" Saving current {label} to filename '{filename}'")
        if df :
            data.to_pickle(os.path.join(data_dir,filename) )
        else:
            pickle.dump(data, open( os.path.join(data_dir,filename), "wb" ))
    else:
        if os.path.isfile(current_filename) and not rewrite :
            print(f" ** Warning: {label} file '{filename}' already exists, saving is not possible **\n ** in directory '{data_dir}'\n ** You can authorize rewriting saying True to 'rewrite' option.\n")

print(f"\nEcriture sous forme de dictionaire, pour INVERSION, de:\n"+\
      f" - tableaux d'indices,\n"+\
      f" - liste du nombre de forcages par modèle,\n"+\
      f" - liste de modeles,\n"+\
      f" - liste de forcages:\n")

print(f"\nEcriture des tableaux d'indices et du nombre de forcages par modèle:\n")

if save_data_ok :

    inversion_suffix = f"{data_gener_method}-S{seed_for_combi}-NMx{n_sim_max_by_mod}_z{region}v0"

    current_inversion_dic = { 'index' : inversion_combi_index,
                              'n_by_mod' : n_inversion_combi,
                              'models' : model_names,
                              'forcings' : forcing_names,
                              'years' : years }

    current_inversion_combi_index_filename = f"inversion-combi-dic_{inversion_suffix}.p"
    
    save_data_file(current_inversion_dic, current_inversion_combi_index_filename, data_dir=data_dir,
                   label='inversion combi dictionary', rewrite=rewrite_files_ok)

else:
    print(f"\n ** data saving not authorized: set True to `save_data_ok` anBoolean variable **\n")




Ecriture sous forme de dictionaire, pour INVERSION, de:
 - tableaux d'indices,
 - liste du nombre de forcages par modèle,
 - liste de modeles,
 - liste de forcages:


Ecriture des tableaux d'indices et du nombre de forcages par modèle:

 Saving current inversion combi dictionary to filename 'inversion-combi-dic_INVDGM1-S1-NMx100_z48v0.p'


In [14]:
save_anc_data_only_ok = False

print(f"\nEcriture des donnees des forcages:\n")

if save_data_ok and not save_anc_data_only_ok:

    for iforcing,forcing in enumerate(forcing_names) :
        print(f"\n{forcing.upper()}:")
        
        current_t_df_varname = f'T_{forcing.lower()}_df'
        xxx_df = globals()[current_t_df_varname]

        current_inversion_index = inversion_combi_index[:,iforcing]

        if dataframe_format_ok :
            current_inversion_filename = f"inversion-{forcing.lower()}_{inversion_suffix}_df.p"

            inversion_df = xxx_df.loc[current_inversion_index]
            inversion_df.reset_index(inplace=True, drop=True)

            save_data_file(inversion_df, current_inversion_filename, data_dir=data_dir,
                           label=f'inversion {forcing.upper()} forcing df', df=True,
                           rewrite=rewrite_files_ok)

        if array_format_ok :
            current_inversion_filename = f"inversion-{forcing.lower()}_{inversion_suffix}.p"

            inversion_array = xxx_df.loc[current_inversion_index].drop('model', axis=1).values
            save_data_file(inversion_array, current_inversion_filename, data_dir=data_dir,
                           label=f'inversion {forcing.upper()} forcing array', rewrite=rewrite_files_ok)

            print(f" inversion {forcing.upper()} array -> dim: {inversion_array.shape}")

elif save_data_ok and save_anc_data_only_ok:
    print(f" ** saving ancillary data only, data arrays not authorized to be saved: set False to `save_anc_data_only_ok` Boolean variables **\n")

else:
    print(f" ** data saving not authorized: set True to `save_data_ok` and False to `save_anc_data_only_ok` Boolean variables **\n")


Ecriture des donnees des forcages:


GHG:
 Saving current inversion GHG forcing df to filename 'inversion-ghg_INVDGM1-S1-NMx100_z48v0_df.p'

AER:
 Saving current inversion AER forcing df to filename 'inversion-aer_INVDGM1-S1-NMx100_z48v0_df.p'

NAT:
 Saving current inversion NAT forcing df to filename 'inversion-nat_INVDGM1-S1-NMx100_z48v0_df.p'


In [15]:
data_dir

'/usr/home/habbar/Bureau/data_nc/stagelong/projetlong/data_source_dr/Region48'

In [16]:
print(f"\nFIN d'Execution:\n  case_out_dir ......... {inversion_suffix}\n  Date/heure d'arret ... {datetime.datetime.now()}\n\n  -> Ready for {n_sim_max_by_mod} n_sim_max_by_mod")


FIN d'Execution:
  case_out_dir ......... INVDGM1-S1-NMx100_z48v0
  Date/heure d'arret ... 2023-07-06 16:56:31.488674

  -> Ready for 100 n_sim_max_by_mod
